In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 

In [2]:
# Carregando dados do filmes
filmes = pd.read_csv('movies.csv')
filmes.columns=['filmeId','titulo','geneross']
filmes.set_index('filmeId', inplace=True)
display(filmes.head())
display(filmes.tail())


,titulo,geneross
filmeId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


,titulo,geneross
filmeId,,
193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
193585,Flint (2017),Drama
193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation
193609,Andrew Dice Clay: Dice Rules (1991),Comedy


In [3]:
# Carregando dados das classificações
notas = pd.read_csv('ratings.csv')
notas.columns=['usuarioId', 'filmeId', 'nota', 'momento']
display(notas.head())
display(notas.tail())

,usuarioId,filmeId,nota,momento
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


,usuarioId,filmeId,nota,momento
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352
100835,610,170875,3.0,1493846415


In [4]:
notas.describe()

,usuarioId,filmeId,nota,momento
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


---
## **Primeira tentativa de recomendação**

Vamos tentar recomendar os filmes de acordo com a quantidade de votos que os filmes receberam

In [5]:
total_de_votos = notas['filmeId'].value_counts()
total_de_votos.head()

356     329
318     317
296     307
593     279
2571    278
Name: filmeId, dtype: int64

In [6]:
filmes.loc[318]

titulo      Shawshank Redemption, The (1994)
geneross                         Crime|Drama
Name: 318, dtype: object

In [7]:
filmes['total_de_votos'] = total_de_votos

In [8]:
filmes.sort_values(by='total_de_votos', ascending=False)

,titulo,geneross,total_de_votos
filmeId,,,
356,Forrest Gump (1994),Comedy|Drama|Romance|War,329.0
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,307.0
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,279.0
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,278.0
...,...,...,...
30892,In the Realms of the Unreal (2004),Animation|Documentary,NaN
32160,Twentieth Century (1934),Comedy,NaN
32371,Call Northside 777 (1948),Crime|Drama|Film-Noir,NaN


In [9]:
notas_medias = notas.groupby(['filmeId']).mean()['nota']
notas_medias.head()

filmeId
1    3.920930
2    3.431818
3    3.259615
4    2.357143
5    3.071429
Name: nota, dtype: float64

In [10]:
# Criando coluna com nota média do filme
filmes['nota_media'] = notas_medias

In [11]:
filmes.head(10)

,titulo,geneross,total_de_votos,nota_media
filmeId,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215.0,3.920930
2,Jumanji (1995),Adventure|Children|Fantasy,110.0,3.431818
3,Grumpier Old Men (1995),Comedy|Romance,52.0,3.259615
4,Waiting to Exhale (1995),Comedy|Drama|Romance,7.0,2.357143
5,Father of the Bride Part II (1995),Comedy,49.0,3.071429
6,Heat (1995),Action|Crime|Thriller,102.0,3.946078
7,Sabrina (1995),Comedy|Romance,54.0,3.185185
8,Tom and Huck (1995),Adventure|Children,8.0,2.875000
9,Sudden Death (1995),Action,16.0,3.125000


In [12]:
# Selecionando as recomendações para ver a média das notas
filmes.sort_values(by='total_de_votos', ascending=False).head(10)

,titulo,geneross,total_de_votos,nota_media
filmeId,,,,
356,Forrest Gump (1994),Comedy|Drama|Romance|War,329.0,4.164134
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,307.0,4.197068
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,279.0,4.161290
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,278.0,4.192446
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,251.0,4.231076
480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,238.0,3.750000
110,Braveheart (1995),Action|Drama|War,237.0,4.031646
589,Terminator 2: Judgment Day (1991),Action|Sci-Fi,224.0,3.970982


In [13]:
# Recomendando com base no valor médio da avaliação
# Veja que aqui temos outro problema. Um filme que tem apenas uma nota, pode representar algum consenço.
# É preciso que alem de uma boa nota, o filme contenha tambem, muitas recomendações
filmes.sort_values(by='nota_media', ascending=False).head(10)

,titulo,geneross,total_de_votos,nota_media
filmeId,,,,
88448,Paper Birds (Pájaros de papel) (2010),Comedy|Drama,1.0,5.0
100556,"Act of Killing, The (2012)",Documentary,1.0,5.0
143031,Jump In! (2007),Comedy|Drama|Romance,1.0,5.0
143511,Human (2015),Documentary,1.0,5.0
143559,L.A. Slasher (2015),Comedy|Crime|Fantasy,1.0,5.0
6201,Lady Jane (1986),Drama|Romance,1.0,5.0
102217,Bill Hicks: Revelations (1993),Comedy,1.0,5.0
102084,Justice League: Doom (2012),Action|Animation|Fantasy,1.0,5.0
6192,Open Hearts (Elsker dig for evigt) (2002),Romance,1.0,5.0


In [14]:
# Selecionando filmes com boa média e quantidade de votos maior do que 50
filmes.query('total_de_votos > 50').sort_values(by='nota_media', ascending=False).head(5)

,titulo,geneross,total_de_votos,nota_media
filmeId,,,,
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
858,"Godfather, The (1972)",Crime|Drama,192.0,4.289062
2959,Fight Club (1999),Action|Crime|Drama|Thriller,218.0,4.272936
1276,Cool Hand Luke (1967),Drama,57.0,4.271930
750,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War,97.0,4.268041


In [15]:
filmes.query('geneross =="Comedy|War" and total_de_votos > 50').sort_values(by='nota_media', ascending=False)

,titulo,geneross,total_de_votos,nota_media
filmeId,,,,
750,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War,97.0,4.268041


## **Calculando a Distancia entre Dois Vetores**

In [16]:
# Função para calcular a distancia entre dois vetores utilizando o Numpy
def distancia_de_vetores(a,b):
    return np.linalg.norm(a-b)

In [17]:
# Filtro por usuário e pegando apenas duas colunas da tabela
def notas_do_usuario(usuario):
    notas_do_usuario = notas.query('usuarioId == @usuario')[['filmeId', 'nota']]
    notas_do_usuario.set_index('filmeId', inplace= True)
    return notas_do_usuario

In [18]:
usuario1 = notas_do_usuario(1)
usuario4 =  notas_do_usuario(4)

display(usuario1.head())
display(usuario4.head())

,nota
filmeId,
1,4.0
3,4.0
6,4.0
47,5.0
50,5.0


,nota
filmeId,
21,3.0
32,2.0
45,3.0
47,2.0
52,3.0


In [19]:
# Passando parametros para resolver problema de overlaping quando tenho duas colunas com mesmo nome em DataFrames diferentes
junto  = usuario1.join(usuario4, lsuffix='_esquerda', rsuffix= '_direita')
# Dropando dados inválidos. Tenho que pegar apenas dados contidos nas duas colunas
junto.dropna(inplace=True)
junto

,nota_esquerda,nota_direita
filmeId,,
47,5.0,2.0
235,4.0,2.0
260,5.0,5.0
296,3.0,1.0
441,4.0,1.0
457,5.0,5.0
553,5.0,2.0
593,4.0,5.0
608,5.0,5.0


In [20]:
distancia_de_vetores(junto['nota_esquerda'], junto['nota_direita'])

11.135528725660043

In [21]:
# Criando uma função que já calcula a deferença entre dois usuários
def distancia_de_usuarios(usuario_id1, usuario_id2):
    nota1 = notas_do_usuario(usuario_id1)
    nota2 = notas_do_usuario(usuario_id2)
    
    diferencas =  nota1.join(nota2, lsuffix='_esquerda', rsuffix= '_direita')
    diferencas.dropna(inplace=True)
    
    return distancia_de_vetores(diferencas['nota_esquerda'], diferencas['nota_direita'])
    

In [22]:
distancia_de_usuarios(1,4)

11.135528725660043

In [23]:
# Verificando a quantidade de usuários unicos no DataFrame
# Não é a quantidade de votos. Posso ter usuários que votaram mais de uma vez.
notas['usuarioId'].nunique()

610

In [41]:
distancias = []

def distancia_de_todos(voce_id):
    for i in notas['usuarioId'].unique():
        informacoes = distancia_de_usuarios(voce_id, i)
        distancias.append([voce_id, i, informacoes])
    return distancias

In [42]:
distancia_de_todos(1)

[0.0, 1.4142135623730951, 8.200609733428363, 11.135528725660043, 3.7416573867739413, 8.602325267042627, 7.106335201775948, 3.7416573867739413, 1.0, 7.858116822750856, 4.242640687119285, 0.7071067811865476, 1.4142135623730951, 5.0, 5.852349955359813, 7.22841614740048, 4.6097722286464435, 8.426149773176359, 17.60681686165901, 6.48074069840786, 9.096702699330127, 5.244044240850758, 5.408326913195984, 4.415880433163924, 1.4142135623730951, 2.23606797749979, 9.219544457292887, 14.882876066137216, 4.55521678957215, 1.8027756377319946, 4.58257569495584, 5.196152422706632, 5.916079783099616, 6.363961030678928, 2.23606797749979, 6.519202405202649, 4.0, 5.5677643628300215, 11.090536506409418, 6.855654600401044, 10.198039027185569, 12.0, 4.795831523312719, 3.7416573867739413, 10.653637876331258, 3.3166247903554, 6.244997998398398, 4.123105625617661, 1.0, 7.33143914930759, 10.476163419878482, 2.5495097567963922, 1.0, 4.123105625617661, 6.5383484153110105, 3.3166247903554, 13.19090595827292, 4.8989